In [5]:

import base64
import io
import math
import operator
import random
import time
import urllib



In [6]:
import matplotlib.pyplot as plt


In [7]:
from PIL import Image


In [8]:
import rtree

In [9]:
import shapefile


In [10]:
from shapely.geometry import Polygon, Point


In [11]:
import pyproj


In [12]:
def slice_at(plist, indices):
    '''
    Slices the list ll at the indices indicated.
    '''
    indices.append(len(plist))
    for fro, to in zip(indices, indices[1:]):
        yield plist[fro:to]

In [13]:

def plot_country(name, parts, center, radius):
    fig, ax = plt.subplots(figsize=(1.8, 1.8))

    circle = plt.Circle(center, radius, color='k', fill=False)
    for part in parts:
        #color = random.choice('bgrcmk')
        color = 'k'
        for point1, point2 in zip(part, part[1:]):
            ax.plot(
                [point1[0], point2[0]],
                [point1[1], point2[1]],
                color+'-')
    ax.add_artist(circle)
    ax.set_aspect('equal', 'datalim')
    imgdata = io.BytesIO()
    #ax.set_title(name)
    plt.axis('off')
    #plt.savefig(name.lower().replace(' ', '-') + '.png', transparent=True)
    #plt.show()
    plt.savefig('tmp.png', format='png', transparent=True)
    img = Image.open('tmp.png')
    png_info = img.info
    img.convert('P').save(imgdata, 'PNG', **png_info)
    imgdata.seek(0)
    plt.close(fig)
    return imgdata

In [14]:
def neighbours(center, radius, delta):
    cmov = delta
    rmov = delta
    cms = [cmov*20, cmov, 0, -cmov, -cmov*20]
    rms = [rmov*20, rmov, 0, -rmov, -rmov*20]
    for cx_mov in cms:
        for cy_mov in cms:
            for r_mov in rms:
                if radius + r_mov > 0:
                    yield ((center[0] + cx_mov, center[1] + cy_mov),
                           radius + r_mov)

In [15]:
def maxmin(arr):
    _max = arr[0]
    _min = arr[0]
    for x in arr:
        if x > _max:
            _max = x
        elif x < _min:
            _min = x
    return _max, _min

In [16]:

def heuristic(polygons, index, c, r, delta, iters=200):
    best_coeff = 0
    parts_area = sum(map(lambda x: x.area, polygons))
    is_ = 0
    for _ in range(iters):
        is_ += 1
        mejoro = False
        for c_, r_ in neighbours(c, r, delta):
            circle = Point(c_).buffer(r_)
            indices = range(len(polygons))
            if circle.bounds != ():
                indices = [int(i) for i in index.intersection(circle.bounds)]
            else:
                print('WARNING: circle.bounds == (). '\
                      'center = %s, radius = %.3f' % (str(c_), r_))
            intersection_area = sum(
                [polygons[i].intersection(circle).area for i in indices])
            coeff = intersection_area / max(math.pi * r_ * r_, parts_area)
            #print(c_, r_)
            #print(intersection_area, circle_area, parts_area)
            if coeff > best_coeff:
                mejoro = True
                c = c_
                r = r_
                best_coeff = coeff
        if not mejoro:
            break
    return c, r, best_coeff, is_

In [17]:
def circle_inside(xmin, ymin, xmax, ymax):
    center = [(xmin + xmax) / 2, (ymin + ymax) / 2]
    radius = min((xmax - xmin) / 2, (ymax - ymin) / 2)
    return center, radius

In [18]:
def circle_outside(xmin, ymin, xmax, ymax):
    center = [(xmin + xmax) / 2, (ymin + ymax) / 2]
    radius = math.sqrt((xmax - xmin)**2.0 + (ymax - ymin)**2.0) / 2
    return center, radius

In [19]:

def analyze_both_circles(polys, index, min_x, min_y, max_x, max_y):
    cinside, rinside = circle_inside(min_x, min_y, max_x, max_y)
    coutside, routside = circle_outside(min_x, min_y, max_x, max_y)
    c_i, r_i, best_i, iin = heuristic(
        polys, index, cinside, rinside, rinside / 200)
    print('[%d, ' % iin, end='', flush=True)
    c_o, r_o, best_o, iout = heuristic(
        polys, index, coutside, routside, routside / 200)
    print('%d]' % iout)
    best_coeff_ = max(best_i, best_o)
    center_ = c_i if best_i > best_o else c_o
    radius_ = r_i if best_i > best_o else r_o
    return center_, radius_, best_coeff_

In [20]:

def find_best_circle(parts, minx, miny, maxx, maxy):
    polys = [Polygon(part).buffer(0) for part in parts]
    index = rtree.index.Index()
    for i, poly in enumerate(polys):
        index.insert(i, poly.bounds)
    print('Analyzing best overall circle...  Iterations: ', end='', flush=True)
    c, r, best_coeff = analyze_both_circles(
        polys, index, minx, miny, maxx, maxy)
    best_polygons = []
    if len(polys) > 1:
        polygons_area = sorted(
            [(poly, poly.area) for poly in polys],
            key=lambda x: x[1], reverse=True)
        areas = list(map(lambda r: r[1], polygons_area))
        if areas[0] > sum(areas[1:]):
            best_polygons = [polygons_area[0][0]]
        else:
            best_polygons = list(map(lambda x: x[0], polygons_area))[:5]
    i = 0
    for part in best_polygons:
        if len(polys) == 1:
            break
        i += 1
        print('Analyzing best part circle... (%d out of %d) Iterations: ' \
            % (i, len(polys)), end='', flush=True)
        x, y = part.exterior.coords.xy
        max_x, min_x = maxmin(x)
        max_y, min_y = maxmin(y)
        center_, radius_, best_coeff_ = analyze_both_circles(
            polys, index, min_x, min_y, max_x, max_y)
        if best_coeff_ > best_coeff:
            c = center_
            r = radius_
            best_coeff = best_coeff_
    # fine tune it.
    print('Fine-tuning the circle... Iterations: ', end='', flush=True)
    c, r, best_coeff, iters = heuristic(polys, index, c, r, r / 1000, 1000)
    print('[%d]' % iters)
    return c, r, best_coeff


In [21]:
def tuple2list(t):
    t = tuple(t)[0]
    return [t[0], t[1]]

In [22]:
def concat(ls):
    return [j for i in ls for j in i]

In [23]:

def mean(ls):
    return float(sum(ls)) / len(ls)

In [24]:
INPUT_FILE = 'LA'
FROM_PROJ = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'


In [25]:
def main():
    countries = shapefile.Reader(INPUT_FILE).shapeRecords()
    results = {}

In [26]:
fields =shapefile.Reader(INPUT_FILE).fields[1:] 
field_names = [field[0] for field in fields] 
# construction of a dctionary field_name:value  
for r in shapefile.Reader(INPUT_FILE).shapeRecords():  
     atr = dict(zip(field_names, r.record))  


In [27]:
print(field_names)

['objectid', 'lad17cd', 'lad17nm', 'lad17nmw', 'bng_e', 'bng_n', 'long', 'lat', 'st_areasha', 'st_lengths']


In [28]:
country = field_names[2]

In [29]:
print(country)

lad17nm


In [74]:

def main():
    countries = shapefile.Reader(INPUT_FILE).shapeRecords()
    results = {}
#     print(countries[8])
    for country in countries:
        t0 = time.process_time()
        name = country.record[1]
        print(country.record[2],country.record[8])
#         if name != 'Nauru': continue
#         print('==== %s (%d -> %d) ====' % (
#             name, len(country.shape.parts), len(country.shape.points)))
#         points = list(map(lambda x: x, country.shape.points))

#         parts_ = list(country.shape.parts) + [len(country.shape.points)]
#         largest_part = (0, 1)
#         for x1, x2 in zip(parts_, parts_[1:]):
#             if x2 - x1 > largest_part[1] - largest_part[0]:
#                 largest_part = (x1, x2)
#         print(parts_, largest_part)

#         # Get the random points from the largest part.
#         random_points = list(
#             map(
#                 list,
#                 random.sample(points[largest_part[0]:largest_part[1]], 2)))

#         # Get the middle points between them.
#         lon, lat = list(
#             pyproj.Geod(ellps='WGS84').npts(
#                 random_points[0][0], random_points[0][1],
#                 random_points[1][0], random_points[1][1],
#                 1)  # Just 1 point in-between them.
#             )[0]

# #         to_proj = '+proj=aeqd +R=6371000 +lat_0=%.2f +lon_0=%.2f ' \
# #                   '+no_defs' % (lat, lon)
# #         print('Projection: %s' % to_proj)

# #         transform = lambda g: pyproj.transform(
# #             pyproj.Proj(FROM_PROJ),
# #             pyproj.Proj(to_proj),
# #             g[0], g[1])
# #         points = list(map(transform, points))

#         parts = list(slice_at(points, country.shape.parts))
#         max_xs, min_xs = maxmin(list(map(lambda t: t[0], points)))
#         max_ys, min_ys = maxmin(list(map(lambda t: t[1], points)))

#         center, radius, best_coeff = find_best_circle(
#             parts, min_xs, min_ys, max_xs, max_ys)

#         print('Time to compute roundness: %.3fs.' % (time.process_time() - t0))
#         print('Result info: %f (%.4f, %.4f) %.4f' \
#             % (best_coeff, center[0], center[1], radius))
#         uri = 'data:image/png;base64,' + urllib.parse.quote(
#             base64.b64encode(
#                 plot_country(name, parts, center, radius).getvalue()))
#         print('Finished plotting %s.' % name)
#         results[name] = (best_coeff, '<img src="%s" />' % uri)

#     f = open('table', 'w')
#     # Does python use theorems for free?
#     # (https://www.mpi-sws.org/~dreyer/tor/papers/wadler.pdf)
#     f.write(
#         '\n'.join(
#             map(lambda s: ' | '.join(map(str, s)),
#                 map(lambda r: [r[0]+1, r[1][0],
#                                '%.3f' % r[1][1][0],
#                                r[1][1][1]],
#                     enumerate(
#                         sorted(results.items(),
#                                key=lambda z: (operator.itemgetter(1)(z))[0],
#                                reverse=True))))))
#     f.close()

In [75]:
main()

Hartlepool 93597464.68468937
Middlesbrough 53878997.02376718
Redcar and Cleveland 244838828.21981904
Stockton-on-Tees 204935660.2773739
Kingston upon Hull, City of 71455773.07550067
East Riding of Yorkshire 2406546932.6630254
Luton 43359037.994624525
Darlington 197482015.73086807
Halton 79111882.40220758
Leicester 73376543.80179568
Warrington 180619479.60694093
Blackburn with Darwen 137057232.83889464
Blackpool 34877461.41961127
North East Lincolnshire 191914627.32571232
York 271938194.4427743
North Lincolnshire 846320971.8646512
Derby 78022267.07255924
Rutland 393743420.7168572
Nottingham 74601634.80932774
Herefordshire, County of 2179677402.4309106
Telford and Wrekin 290324518.12275
Stoke-on-Trent 93416841.66116361
Bath and North East Somerset 351163380.8022364
Southend-on-Sea 41751922.20483953
Bristol, City of 109720694.01738296
Torbay 62936711.84336812
North Somerset 374629904.2975286
Bournemouth 46204494.13853425
South Gloucestershire 496933994.36149293
Slough 32538450.312692445
P

In [ ]:
Hartlepool 93597464.68468937
Middlesbrough 53878997.02376718
Redcar and Cleveland 244838828.21981904
Stockton-on-Tees 204935660.2773739
Kingston upon Hull, City of 71455773.07550067
East Riding of Yorkshire 2406546932.6630254
Luton 43359037.994624525
Darlington 197482015.73086807
Halton 79111882.40220758
Leicester 73376543.80179568
Warrington 180619479.60694093
Blackburn with Darwen 137057232.83889464
Blackpool 34877461.41961127
North East Lincolnshire 191914627.32571232
York 271938194.4427743
North Lincolnshire 846320971.8646512
Derby 78022267.07255924
Rutland 393743420.7168572
Nottingham 74601634.80932774
Herefordshire, County of 2179677402.4309106
Telford and Wrekin 290324518.12275
Stoke-on-Trent 93416841.66116361
Bath and North East Somerset 351163380.8022364
Southend-on-Sea 41751922.20483953
Bristol, City of 109720694.01738296
Torbay 62936711.84336812
North Somerset 374629904.2975286
Bournemouth 46204494.13853425
South Gloucestershire 496933994.36149293
Slough 32538450.312692445
Plymouth 79840318.05192322
Poole 64745074.84816722
Swindon 230091709.05649447
Peterborough 343423850.82824844
Thurrock 163438914.35679573
Medway 193485251.47192115
Bracknell Forest 109346432.9503872
Windsor and Maidenhead 198421643.55182135
West Berkshire 704162839.3027713
Reading 40384392.86702668
Wokingham 178974516.0041921
Milton Keynes 308649080.15699625
Brighton and Hove 82815432.46858159
Portsmouth 40359257.95119152
Southampton 49901083.29740158
Isle of Wight 380221619.0531368
County Durham 2231575405.91365
Cheshire East 1166350896.1760626
Cheshire West and Chester 916656233.831345
Shropshire 3197310396.649548
Cornwall 3549517292.6082683
Isles of Scilly 16387719.473811384
Wiltshire 3255372681.540217
South Tyneside 64415615.31562227
Bedford 476375539.39235973
Chiltern 196376836.14714676
Brent 43223381.06081882
Central Bedfordshire 715580550.8672259
Northumberland 5026317325.468403
Aylesbury Vale 902687606.0663174
South Bucks 141276825.5163013
Wycombe 324627075.55395985
Cambridge 40705205.21673466
East Cambridgeshire 651314504.4851465
Christchurch 50401724.27065952
Fenland 546479193.8817092
Bolsover 160352391.01461327
Huntingdonshire 912556058.4645547
Eastbourne 44154188.955269784
South Cambridgeshire 901700178.3218403
Allerdale 1257852989.7038176
Barrow-in-Furness 77964664.81448936
Carlisle 1039179105.059753
Copeland 737700721.1729326
Chesterfield 66027831.50399012
Derbyshire Dales 795432890.4325411
Eden 2156542958.3297124
South Lakeland 1552952321.0821369
Amber Valley 265411696.60774463
Erewash 109630992.94128188
High Peak 540226441.2380769
Exeter 47041656.55158964
Mid Devon 912825356.9518803
North Devon 1085979379.4876442
South Hams 886568922.1885623
North East Derbyshire 275665117.15790313
Hastings 29725810.158835433
South Derbyshire 338094360.94028
Southwark 28845353.226642914
East Devon 814490163.8151479
Teignbridge 673911035.1768317
Torridge 985367085.2526011
West Devon 1161047643.5121017
East Dorset 354430963.3167467
Basildon 110014693.5936593
North Dorset 609270414.5530019
Purbeck 404382765.8555233
West Dorset 1081656830.2548907
Weymouth and Portland 41824037.59497423
Lewes 292070616.11172587
Coventry 98624085.35783228
Rother 511762615.6125473
Wealden 834968660.6340317
Sutton 43835897.91422358
Braintree 611699944.8970371
Tower Hamlets 19781602.10447551
Brentwood 153140702.09700543
Castle Point 45041292.77385386
Fareham 74256855.210263
Chelmsford 342267936.02088267
Gosport 25363251.114171326
Waltham Forest 38821994.582402
Colchester 333130561.04524314
Epping Forest 338967946.9405725
Harlow 30530204.010974776
Maldon 358586632.3969514
Bromley 150125317.3759654
Rochford 169346280.4535167
Tendring 337764521.7440999
Uttlesford 641112303.2130463
Cheltenham 46568222.5614899
Cotswold 1164545369.8306437
Forest of Dean 526308804.464256
Gloucester 40547180.03133927
Stroud 460707303.6294944
Camden 21807916.50195192
Tewkesbury 414402984.2471393
Wandsworth 34264444.58544471
Basingstoke and Deane 633794445.8675013
Hart 215260519.7162902
East Hampshire 514431718.94425493
Eastleigh 79786712.19913639
Havant 55337667.29428494
New Forest 752999033.4753219
Rushmoor 39050958.38221338
Watford 21399455.86804067
Test Valley 627610844.693666
Burnley 110689853.87130257
Winchester 660968527.2147442
Broxbourne 51459623.75413508
Westminster 21459012.22725562
Dacorum 212450092.6145219
Hertsmere 101117185.78807506
Hyndburn 73004074.21922185
Lancaster 576119067.6189798
North Hertfordshire 375374328.2355243
Three Rivers 88805059.09300072
Ashford 580584769.7384119
Pendle 169367124.4680651
Canterbury 308910023.57886124
Dartford 72768920.6436615
Dover 315064333.73407835
Gravesham 99057271.19772299
Maidstone 393323793.68316376
Preston 142288117.28695637
Sevenoaks 370332695.8482601
Welwyn Hatfield 129481548.83923224
Shepway 356809471.7740277
Swale 374406597.3823092
Thanet 103374176.77494372
Tonbridge and Malling 240103385.71451056
Croydon 86535674.74708962
Tunbridge Wells 331304607.0125047
Chorley 202745813.73364288
Fylde 165716545.75713995
Ribble Valley 584451506.6005602
Rossendale 138019593.04968515
South Ribble 112990105.06676714
Blaby 130478802.62398566
West Lancashire 346754809.3757495
Ealing 55538606.36217018
Wyre 282052816.32058513
Charnwood 279022966.4599767
Hinckley and Bosworth 297357217.0225525
Harborough 592724627.2850673
Melton 481360284.31368095
Corby 80269876.05431432
North West Leicestershire 279328063.96375555
Oadby and Wigston 23517259.057276256
Enfield 82171315.91022512
Boston 364852630.087722
East Lindsey 1764963537.812935
Lincoln 35650912.81250873
Wellingborough 163099452.86931917
North Kesteven 922440516.6377527
Ashfield 109548883.2539217
South Holland 750790937.4990181
Gedling 119962355.633777
South Kesteven 942602113.9036157
Mansfield 76668404.95215604
West Lindsey 1155679841.3071582
Oxford 45620581.49580139
Breckland 1305111487.2845232
Epsom and Ewell 34119022.68579887
Broadland 552378122.0581568
Great Yarmouth 174429137.05838197
King's Lynn and West Norfolk 1438815038.9930546
North Norfolk 964002582.4708716
Norwich 39011534.46437117
South Norfolk 907643127.8528581
Daventry 665646793.3738682
East Northamptonshire 509714857.1765501
Runnymede 78052699.75323586
Kettering 233508322.45488176
Northampton 80731919.98998997
South Northamptonshire 634079976.4582683
Spelthorne 51167077.41942948
Craven 1178932805.4848151
Hambleton 1311245281.8354902
Surrey Heath 95077255.39162984
Harrogate 1309028122.5396514
Richmondshire 1318709575.481483
Greenwich 47305858.03433148
Ryedale 1506634623.220653
Hackney 19055506.491199724
Scarborough 816557157.4918017
Selby 599308223.3267715
Bassetlaw 637847029.6384604
Broxtowe 80138662.73885849
Newark and Sherwood 651367244.3991212
Hammersmith and Fulham 16382501.53489421
Haringey 29601369.609379694
Rushcliffe 409239490.2775473
Tandridge 248195896.93203735
Harrow 50448756.66243362
Cherwell 588715500.0169953
South Oxfordshire 678433682.7617737
Vale of White Horse 578683149.8168879
West Oxfordshire 714423758.198226
Havering 112341836.6083252
Mendip 739393243.442036
Sedgemoor 564360352.1602123
South Somerset 959011931.383081
Taunton Deane 462318693.4570731
Hillingdon 115737806.57007936
West Somerset 727011011.5945923
Cannock Chase 78893044.13941012
East Staffordshire 389989026.2076492
Lichfield 331257810.4479276
Manchester 115595841.65213732
Newcastle-under-Lyme 210962774.58439007
Oldham 142343448.12524652
South Staffordshire 407322539.1932942
Stafford 598191550.8677891
Staffordshire Moorlands 575824949.204152
Tamworth 30861286.847729016
Babergh 595279274.5567063
Forest Heath 377760642.9901555
Ipswich 39531192.34876834
Hounslow 55972269.95534084
Mid Suffolk 871036432.5860236
Islington 14837420.45185494
St Edmundsbury 656961006.9901007
Suffolk Coastal 891723806.5506648
Waveney 370101416.7868216
Elmbridge 96297935.43301444
Guildford 270995199.2592339
Rochdale 158161475.68368715
Mole Valley 258304538.68003416
Reigate and Banstead 129127877.91150616
Kensington and Chelsea 12139191.854274275
Waverley 345188766.45027965
Woking 63560964.13336274
North Warwickshire 284266836.63339967
Nuneaton and Bedworth 78974918.77465221
Salford 97158283.14985467
Rugby 353519324.48383105
Stratford-on-Avon 977881217.5067726
Wyre Forest 195421909.28116155
Warwick 282877698.9717242
Adur 41953113.95743161
Kingston upon Thames 37275479.538196184
Arun 220930942.1845398
Chichester 786243227.5690584
Crawley 44969484.73616564
Lambeth 26797935.797779236
Horsham 530260696.19249624
Mid Sussex 333996393.81195307
Worthing 32509668.319268215
St Albans 161267076.11824048
Bromsgrove 217020855.3291982
Malvern Hills 577081295.564987
Redditch 54266227.10258322
Worcester 33295694.274359606
Wychavon 663449161.6580256
East Hertfordshire 475708246.2256568
Stevenage 25941961.459872615
St. Helens 136413423.8479852
Bolton 139795200.5703215
Bury 99463149.79594226
Lewisham 35188851.36830722
Stockport 126048310.33670594
Tameside 103165545.45810802
Trafford 106023303.7387304
Wigan 188161944.22149703
Knowsley 86537089.4374396
Liverpool 111821887.2918496
Sefton 154939762.87655103
Wirral 157061607.08056954
Merton 37628562.9640046
Barnsley 329132793.93756115
North Tyneside 82296262.55204669
Newham 36200915.20693436
Doncaster 568029182.7042874
Rotherham 286456445.7292262
Sheffield 367862436.5332125
Newcastle upon Tyne 113439029.83394788
Redbridge 56412286.59070986
Sunderland 137385698.554141
Birmingham 267761494.90203372
Dudley 97939613.98417486
Sandwell 85597166.0743599
Richmond upon Thames 57366682.99636612
Solihull 178280176.85965627
Walsall 103998001.49870862
Wolverhampton 69439134.31904683
Bradford 366393462.2951593
Calderdale 363994684.6383717
Kirklees 408496919.1149084
Wakefield 338612310.2323351
Leeds 551788605.8753263
Bexley 60587802.8170755
Gateshead 142359624.85339847
City of London 2897650.24061696
Barking and Dagenham 36108163.41734254
Barnet 86799645.48826592
Clackmannanshire 158980179.38455287
Dumfries and Galloway 6437731658.683273
East Ayrshire 1270374981.3568156
East Lothian 679254588.1030412
East Renfrewshire 174253413.98240155
Na h-Eileanan Siar 3097130797.6244516
Falkirk 297322868.8992642
Fife 1325181202.1509004
Highland 26161949189.693542
Inverclyde 161932159.4851069
Midlothian 355264446.7304053
Moray 2237811873.235617
North Ayrshire 885534348.7101523
Orkney Islands 1013971666.4391378
Perth and Kinross 5383648544.812105
Scottish Borders 4739222748.26833
Shetland Islands 1468615000.5792663
South Ayrshire 1224062648.4859889
South Lanarkshire 1773926898.2232826
Stirling 2253445632.159051
Aberdeen City 185691104.2851371
Aberdeenshire 6317945679.700755
Argyll and Bute 7007010160.716568
City of Edinburgh 263361509.47013313
West Dunbartonshire 177274358.18483523
Renfrewshire 261462132.5949115
East Dunbartonshire 174481267.67691496
West Lothian 428827217.76868737
Angus 2185174038.2323394
Dundee City 59855429.60727202
North Lanarkshire 472259257.42693
Glasgow City 174724285.23263997
Isle of Anglesey 714480365.1402599
Gwynedd 2548255638.287581
Conwy 1130283556.1998491
Bridgend 250783812.89255312
Denbighshire 838546715.9331274
Flintshire 437492787.31892455
Wrexham 503684324.84540546
Ceredigion 1788994610.2753408
Pembrokeshire 1618598471.510508
Carmarthenshire 2371121436.562757
Swansea 379605290.7509292
Neath Port Talbot 442304884.67240655
Vale of Glamorgan 331179299.64859134
Cardiff 142352815.94899976
Rhondda Cynon Taf 424134185.8121676
Torfaen 126223130.72751997
Caerphilly 277352304.5341038
Blaenau Gwent 108725534.8398173
Merthyr Tydfil 111930294.41241744
Monmouthshire 850268570.1731719
Newport 190506626.04560328
Powys 5195426136.895562